# Exploring dataset & Database creation

## Contents
* [<font size=4>Exploring dataset</font>](#cell_01)
* [<font size=4>Database creation</font>](#cell_02)
* [<font size=4>Data load</font>](#cell_03)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Exploring dataset <a id="#cell_01"></a>

### Interactions

The file contains information about user-book interactions

In [2]:
df1 = pd.read_csv('Datasets/interactions.csv')
df1.head()

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01
2,66991,198453,89,NaN,2018-01-01
3,46791,83486,23,5.0,2018-01-01
4,79313,188770,88,5.0,2018-01-01


In [3]:
# Interactions count
df1.shape[0]

1533078

In [4]:
# Rating values
df1['rating'].unique()

array([nan,  5.,  4.,  2.,  3.,  1.])

### User features

In [6]:
df2 = pd.read_csv('Datasets/users.csv')
df2.head()

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0


In [7]:
# Users count
df2.shape[0]

142888

In [8]:
# Sex values
df2['sex'].unique()

array([nan,  0.,  1.])

### Book features

In [9]:
df3 = pd.read_csv('Datasets/items.csv')
df3.head()

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [10]:
# Books count
df3.shape[0]

59599

In [11]:
# Year values
df3['year'].unique()

array(['1886', '1894', '1869', ..., '2004, 2007', '2014–2019',
       '1965,1966,1967,1968'], dtype=object)

## Database creation <a id="#cell_02"></a>

## Data load <a id="#cell_03"></a>

pd 